## Importation de donnnes

In [2]:
data<-read.csv("data_visco.csv")
data_lasso <- read.csv("data_lasso.csv")
data_AIC <- read.csv("data_AIC.csv")


In [3]:
dim(data_lasso)
dim(data_AIC)

[1] 7247   29

[1] 7247   20

## t-IMSE standard

In [4]:
# Charger la librairie
library("KrigInv") #Pour t-IMSE
library("nloptr") #Pour la fonction d'optimisation Cobyla

Le chargement a nécessité le package : DiceKriging



In [5]:
x = data[1:60,1:116]
#x = matrix(sample(5:500, 116*30, replace=TRUE),nrow = 30)
data_reduced = data[1:120,1:116]
#des = data_reduced[c(-1)]
shape = c(dim(data_reduced))
noise = matrix(runif(prod(shape),min = 0.00000001,max = 0.00000002), nrow = 120)
mod = km(formula = ~1, design = data_reduced + noise, response = data_reduced[1])
tmse_optim(x, model = mod, T = 400)


optimisation start
------------------
* estimation method   : MLE 
* optimisation method : BFGS 
* analytical gradient : used
* trend model : ~1
* covariance model : 
  - type :  matern5_2 
  - nugget : NO
  - parameters lower bounds :  1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 
  - parameters upper bounds :  12076 0.78 1.980193e-08 44.86 1.99

[1] 361.8586 361.8586 361.8586 361.8586 361.8586 361.8586 361.8586 361.8586
 [9] 361.8586 361.8586 361.8586 361.8586 361.8586 361.8586 361.8586 361.8586
[17] 361.8586 361.8586 361.8586 361.8586 361.8586 361.8586 361.8586 361.8586
[25] 361.8586 361.8586 361.8586 361.8586 361.8586 361.8586 361.8586 361.8586
[33] 361.8586 361.8586 361.8586 361.8586 361.8586 361.8586 361.8586 361.8586
[41] 361.8586 361.8586 361.8586 361.8586 361.8586 361.8586 361.8586 361.8586
[49] 361.8586 361.8586 361.8586 361.8586 361.8586 361.8586 361.8586 361.8586
[57] 361.8586 361.8586 361.8586 361.8586

In [6]:
fn<-function(x){
  tmse_optim(x, model = mod, T = 30)
}
contr<-function(x){
  h<-numeric(2)
  h[1]<- 100-2*x #g(x)>=0
  h[2]<- x  #-g(x)>=0
  return(h)
}

#cobyla(x[1,1:116],fn,hin=contr)

## Adaptation t-IMSE à nos contraintes

Au lieu d'utiliser la fonction **tmse_optim** standard, il nous faut considerer les contraintes du simplex

In [7]:
#simulation de x unifome sur [0,1] et la somme egale à 1
simul <- function(d)
{
  # set.seed(18)
  u = runif(n=d-1, min=0,max=1)
  #replicate(n=5,u) si on doit tester avec plusieurs vecteurs diff
  u = c(0,u,1)
  u = sort(u)
  x = u[2:(d+1)] - u[1:d]  
  return(x)
}

In [8]:
g = function (x, model, T, method.param = NULL) 
{
    y <- t(x)
    if (ncol(y) == model@d) 
        z <- y
    if (ncol(y) != model@d) 
        z <- x
    krig <- predict_nobias_km(object = model, newdata = as.data.frame(z), 
        type = "UK", se.compute = TRUE)
    mk <- krig$mean
    sk <- krig$sd
    if (is.null(method.param)) 
        method.param <- 0
    epsilon <- method.param
    if (length(T) == 1) {
        W <- 1/sqrt(2 * pi * (sk^2 + epsilon^2)) * exp(-0.5 * 
            ((mk - T)/sqrt(sk^2 + epsilon^2))^2)
    }
    else {
        W0 <- 1/sqrt(2 * pi * (sk^2 + epsilon^2))
        W <- 0
        for (i in 1:length(T)) {
            Ti <- T[i]
            W <- W + W0 * exp(-0.5 * ((mk - Ti)/sqrt(sk^2 + epsilon^2))^2)
        }
    }
    g <- W * sk^2
    return(g)
}

In [9]:
help(tmse_optim)

tmse_optim               package:KrigInv               R Documentation

_T_a_r_g_e_t_e_d _M_S_E _c_r_i_t_e_r_i_o_n

_D_e_s_c_r_i_p_t_i_o_n:

     Evaluation of the Targeted MSE criterion. To be used in
     optimization routines, like in ‘max_infill_criterion’

_U_s_a_g_e:

     tmse_optim(x, model, T, method.param = NULL)
     
_A_r_g_u_m_e_n_t_s:

       x: Input vector at which one wants to evaluate the criterion.
          This argument can be either a vector of size d (for an
          evaluation at a single point) or a p*d matrix (for p
          simultaneous evaluations of the criterion at p different
          points).

   model: An object of class ‘km’ (Kriging model).

       T: Array containing one or several thresholds.

method.param: Scalar tolerance around the targets T.

_V_a_l_u_e:

     targeted MSE value. When the argument ‘x’ is a vector the function
     returns a scalar. When the argument ‘x’ is a p*d matrix the
   

In [10]:
d = 29
N = 10*d
s = 0
data_Lasso_reduced = data_lasso[1:N,]
shape = c(dim(data_Lasso_reduced))
noise = matrix(runif(prod(shape),min = 0.0001,max = 0.0002), nrow = N)
mod = km(formula = ~1, design = data_Lasso_reduced+noise, response = data_Lasso_reduced$vis)
x = data_Lasso_reduced[1:50,]
tmse_optim(x, model = mod, T = 400)



optimisation start
------------------
* estimation method   : MLE 
* optimisation method : BFGS 
* analytical gradient : used
* trend model : ~1
* covariance model : 
  - type :  matern5_2 
  - nugget : NO
  - parameters lower bounds :  1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 
  - parameters upper bounds :  476715169234 0.0001986321 44.86013 149.54 0.9201923 0.0001992104 22.92008 0.6001918 0.0001994919 0.0001995679 0.0001997994 0.08008101 105.1 0.0001965706 57.84002 16.82003 3.000009 0.0001991693 110.7201 0.0001981544 2.000198 0.0001990489 141.5999 0.0001981993 0.6201964 0.0001972447 0.0001987072 10.00017 157.62 
  - best initial criterion value(s) :  -7129.772 

N = 29, M = 5 machine precision = 2.22045e-16
At X0, 0 variables are exactly at the bounds
At iterate     0  f=       7129.8  |proj g|=        4.131
At iterate     1  f =       6918.1  |proj g|

[1] 633884379 633691380 635185982 621515150 633719294 633814254 633707172
 [8] 633539578 627414781 634123993 633562407 634633169 636067128 620614036
[15] 615181685 632537871 629452705 634737626 634862119 631580529 631580529
[22] 630730082 630730082 630278451 631378946 630745669 631019219 631689677
[29] 631144339 630944835 630944835 632685335 602779436 613780549 604982718
[36] 599400897 611526943 608001154 614436288 606195155 620673747 605763242
[43] 620989672 619124794 616569199 630745669 630018305 631185912 630934048
[50] 630934048

In [11]:
tmse <- function(d,mod, T)
{
    s = 0
    for(i in 1:1000){
    x = simul(d) 
    s = s + g(x, model = mod,T)
    }
    tmse = s/1000
    return(tmse)
}
    

In [12]:
print(tmse(d,mod,T=400))

[1] 544921365


In [13]:
x = data_Lasso_reduced[1:50,]
shape = c(dim(data_reduced))
noise = matrix(runif(prod(shape),min = 0.0001,max = 0.0002), nrow = 120)
mod = km(formula = ~1, design = data_Lasso_reduced + noise, response = data_Lasso_reduced[1])




optimisation start
------------------
* estimation method   : MLE 
* optimisation method : BFGS 
* analytical gradient : used
* trend model : ~1
* covariance model : 
  - type :  matern5_2 
  - nugget : NO
  - parameters lower bounds :  1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 
  - parameters upper bounds :  476715169234 0.0001989291 44.86019 149.5401 0.9201948 0.0001983654 22.92013 0.6001685 0.0001999801 0.0001993255 0.0001986367 0.08002562 105.1002 0.0001963732 57.8401 16.82002 3.000007 0.0001983948 110.7201 0.000198544 2.000194 0.0001973981 141.5998 0.0001953732 0.6201676 0.0001981253 0.00019705 10.00017 157.62 
  - best initial criterion value(s) :  -7138.348 

N = 29, M = 5 machine precision = 2.22045e-16
At X0, 0 variables are exactly at the bounds
At iterate     0  f=       7138.3  |proj g|=       1.5717
At iterate     1  f =       6937.6  |proj g

In [14]:
new_tmse = function(x_new){
    
    
    
}

In [16]:
# X = data_Lasso_reduced
# nrow = nrow(X)
# for (i in 1:10){
#     # Loop for individual search
#     noise = matrix(runif(prod(dim(X)),min = 0.0001,max = 0.0002), nrow = nrow)
#     mod = km(formula = ~1, design = X + noise, response = X[1])
#     x_new = cobyla()
    
# }

In [21]:
new_tmse = function(x_new, data, d = 29, niter = 1000){
    # d: dimension, d = 29
    s = 0
    data_new = data.frame(data)
    data_new = rbind(data, x_new)
    noise = matrix(runif(prod(dim(data_new)),min = 0.0001,max = 0.0002), nrow = nrow(data_new))
    mod = km(formula = ~1, design = data_new + noise, response = data_new[1])
    
    for(i in 1:niter){
    x = simul(d)
    s = s + g(x, model = mod, T = 400)
    }
    tmse = s/niter
    return(tmse)    
}

In [19]:
# OPTIMISATION TMSE PROCESS

# Some parameters
d = 29  # data dimension
n_init = 10*d   # initial number of data points
n_points = nrow(data_lasso) # total number of data points
n_adds = 2 # number of additional data points

# Initialize randomly data by index
index = sample(1:nrow(data_lasso),n_init)
X = data_lasso[index, ]

print("Starting...")
# Loop over additional data points
for (i in 1:n_adds){
    # Loop over all data points except the data that are already selected
    print(i)
    argmin = NULL
    tmse = new_tmse(x_new = NULL, data = X)
    for (k in setdiff(1:n_points, index)){
        print("Loop at k = ", k)
        new_tmse = new_tmse(x_new = data_lasso[k,], data = X, niter = 100)
        if (new_tmse < tmse) argmin = k
        
    }
    print(argmin)
    X = rbind(X, data_lasso[argmin,])
    index = c(index, argmin)
}

[1] "Starting..."


In [22]:
# OPTIMISATION TMSE PROCESS

# Some parameters
d = 29  # data dimension
n_init = 10*d   # initial number of data points
n_points = nrow(data_lasso) # total number of data points
n_adds = 2 # number of additional data points

# Initialize randomly data by index
index = sample(1:nrow(data_lasso),n_init)
X = data_lasso[index, ]

print("Starting...")

argmin = NULL
tmse = new_tmse(x_new = NULL, data = X)

[1] "Starting..."

optimisation start
------------------
* estimation method   : MLE 
* optimisation method : BFGS 
* analytical gradient : used
* trend model : ~1
* covariance model : 
  - type :  matern5_2 
  - nugget : NO
  - parameters lower bounds :  1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 1e-10 
  - parameters upper bounds :  1.907611e+12 0.0001986403 80.00001 73.56002 90.00014 74.26018 89.48016 15.56011 0.5381778 115.2401 67.98002 1.760002 51.66008 77.48011 39.30007 39.5601 54.60006 0.0001982339 79.50011 0.0001978923 12.08003 4.280067 158.9799 3.4001 1.000123 59.80001 28.00015 19.20201 157.6199 
  - best initial criterion value(s) :  -7314.788 

N = 29, M = 5 machine precision = 2.22045e-16
At X0, 0 variables are exactly at the bounds
At iterate     0  f=       7314.8  |proj g|=       2.1492
At iterate     1  f =       7106.4  |proj g|=        2.6348
At

In [23]:
print(tmse)

[1] 1964788529
